In [ ]:
# Cell 2: Import các thư viện
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from underthesea import word_tokenize
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import optuna
from datasets import Dataset
from pathlib import Path
import sys
project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [ ]:
data = pd.read_excel(r"E:\Project_NguyenMinhVu_2211110063\Source\datasets\dataset extractive\6000 bài đoc\bai_doc_tom_tat.xlsx")  # Thay bằng đường dẫn file của bạn
DATA = pd.DataFrame(data)
DATA

,id,title,content,Summarizex,output_len,category,topic,source,grade,publisher,summarize_short,summarize_medium,Summarize
0,1.0,Tôi là học sinh lớp 2,"Ngày khai trường đã đến. Sáng sớm, mẹ mới gọi ...",NaN,NaN,NaN,",",NaN,Lớp 2,KẾT NỐI TRI THỨC,Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi ...,Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi ...,Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi ...
1,2.0,Ngày hôm qua đâu rồi?,Em cầm tờ lịch cũ:\n– Ngày hôm qua đâu rồi?\nR...,NaN,NaN,NaN,NaN,NaN,Lớp 2,KẾT NỐI TRI THỨC,– Ngày hôm qua ở lại Trong vở hồng của con Con...,Em cầm tờ lịch cũ: – Ngày hôm qua đâu rồi? – N...,Em cầm tờ lịch cũ: – Ngày hôm qua đâu rồi? – N...
2,3.0,Niềm vui của Bi và Bống,"Khi cơn mưa vừa dứt, hai anh em Bi và Bống chợ...",NaN,NaN,NaN,NaN,NaN,Lớp 2,KẾT NỐI TRI THỨC,"Khi cơn mưa vừa dứt, hai anh em Bi và Bống chợ...","Khi cơn mưa vừa dứt, hai anh em Bi và Bống chợ...","Khi cơn mưa vừa dứt, hai anh em Bi và Bống chợ..."
3,4.0,Làm việc thật và vui,"Quanh ta, mọi vật, mọi người đều làm việc.\nCá...",NaN,NaN,NaN,NaN,NaN,Lớp 2,KẾT NỐI TRI THỨC,"Quanh ta, mọi vật, mọi người đều làm việc. Con...","Quanh ta, mọi vật, mọi người đều làm việc. Con...","Quanh ta, mọi vật, mọi người đều làm việc. Con..."
4,5.0,Em có xinh không?,Voi em thích mặc đẹp và thích được khen xinh. ...,NaN,NaN,NaN,NaN,NaN,Lớp 2,KẾT NỐI TRI THỨC,Voi em thích mặc đẹp và thích được khen xinh. ...,Voi em thích mặc đẹp và thích được khen xinh. ...,Voi em thích mặc đẹp và thích được khen xinh. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6268,NaN,Tứ đại mỹ nhân thời cổ Trung Quốc là những ai?,Lịch sử Trung Quốc có bốn người đẹp làm khuynh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lịch sử Trung Quốc có bốn người đẹp làm khuynh...,Lịch sử Trung Quốc có bốn người đẹp làm khuynh...,Lịch sử Trung Quốc có bốn người đẹp làm khuynh...
6269,NaN,Đạo giáo đã nảy sinh như thế nào?,"Đạo giáo (hay Lão giáo), Phật giáo và Hồi giáo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Đạo giáo (hay Lão giáo), Phật giáo và Hồi giáo...","Đạo giáo (hay Lão giáo), Phật giáo và Hồi giáo...","Đạo giáo (hay Lão giáo), Phật giáo và Hồi giáo..."
6270,NaN,Tại sao bệnh dịch hạch lại trở thành đại hoạ c...,"Ngày 24 tháng 3 năm 1345, người ta phát hiện t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ngày 24 tháng 3 năm 1345, người ta phát hiện t...","Ngày 24 tháng 3 năm 1345, người ta phát hiện t...","Ngày 24 tháng 3 năm 1345, người ta phát hiện t..."
6271,NaN,Đế quốc Ôt-tôman ra đời như thế nào?,"Giữa thế kỷ XIII, Tiểu Á rơi vào tình trạng hỗ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Giữa thế kỷ XIII, Tiểu Á rơi vào tình trạng hỗ...","Giữa thế kỷ XIII, Tiểu Á rơi vào tình trạng hỗ...","Giữa thế kỷ XIII, Tiểu Á rơi vào tình trạng hỗ..."


In [ ]:
Data_grade45 = DATA[DATA['grade'].isin(['Lớp 5', 'Lớp 4'])]
Data_grade3_random = DATA[DATA['grade'].isin(['Lớp 3'])].sample(n=round(len(DATA[DATA['grade'].isin(['Lớp 5'])])) + 1, random_state=42)
Data_grade345 = pd.concat([Data_grade3_random, Data_grade45])
Data_grade345

,id,title,content,Summarizex,output_len,category,topic,source,grade,publisher,summarize_short,summarize_medium,Summarize
391,442.0,Điều kì diệu,Bạn có thấy lạ không \nMỗi đứa mình một khác \...,NaN,NaN,NaN,NaN,NaN,Lớp 4,KẾT NỐI TRI THỨC,Bạn có thấy lạ không Mỗi đứa mình một khác Cùn...,Bạn có thấy lạ không Mỗi đứa mình một khác Cùn...,Bạn có thấy lạ không Mỗi đứa mình một khác Cùn...
392,443.0,Thi nhạc,Hôm nay là ngày thi tốt nghiệp của các học trò...,NaN,NaN,NaN,NaN,NaN,Lớp 4,KẾT NỐI TRI THỨC,Hôm nay là ngày thi tốt nghiệp của các học trò...,Hôm nay là ngày thi tốt nghiệp của các học trò...,Hôm nay là ngày thi tốt nghiệp của các học trò...
393,444.0,Anh em sinh đôi,Khánh và Long là anh em sinh đôi. Hai anh em g...,NaN,NaN,NaN,NaN,NaN,Lớp 4,KẾT NỐI TRI THỨC,Khánh và Long là anh em sinh đôi. Hai anh em m...,Khánh và Long là anh em sinh đôi. Hai anh em m...,Khánh và Long là anh em sinh đôi. Hai anh em m...
394,445.0,Công chúa và người dẫn chuyện,"Sau bữa ăn trưa, cô giáo thông báo Giét-xi đượ...",NaN,NaN,NaN,NaN,NaN,Lớp 4,KẾT NỐI TRI THỨC,"Sau bữa ăn trưa, cô giáo thông báo Giét-xi đượ...","Sau bữa ăn trưa, cô giáo thông báo Giét-xi đượ...","Sau bữa ăn trưa, cô giáo thông báo Giét-xi đượ..."
395,446.0,Thằn lằn xanh và tắc kè,"Một buổi tối nọ, thằn lằn xanh đang bò trên cà...",NaN,NaN,NaN,NaN,NaN,Lớp 4,KẾT NỐI TRI THỨC,"Trong khi đó, tắc kẻ cũng cảm thấy mình không ...","Một buổi tối nọ, thằn lằn xanh đang bò trên cà...","Một buổi tối nọ, thằn lằn xanh đang bò trên cà..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,250.0,Trò chuyện cùng mẹ,Thời gian vui nhất trong buổi tối của Thư và H...,NaN,NaN,NaN,NaN,NaN,Lớp 3,KẾT NỐI TRI THỨC,Thời gian vui nhất trong buổi tối của Thư và H...,Thời gian vui nhất trong buổi tối của Thư và H...,Thời gian vui nhất trong buổi tối của Thư và H...
261,293.0,Ngọn lửa Ô-lim-pích,Tục lệ tổ chức Đại hội Thể thao Ô-lim-pích đã ...,NaN,NaN,NaN,NaN,NaN,Lớp 3,KẾT NỐI TRI THỨC,Tục lệ tổ chức Đại hội Thể thao Ô-lim-pích đã ...,Tục lệ tổ chức Đại hội Thể thao Ô-lim-pích đã ...,Tục lệ tổ chức Đại hội Thể thao Ô-lim-pích đã ...
364,413.0,Sông quê,Gió chiều ru hiền hòa\nRung bờ tre xào xạc\nBầ...,NaN,NaN,NaN,NaN,NaN,Lớp 3,CÁNH DIỀU,Gió chiều ru hiền hòa Rung bờ tre xào xạc Bầy ...,Gió chiều ru hiền hòa Rung bờ tre xào xạc Bầy ...,Gió chiều ru hiền hòa Rung bờ tre xào xạc Bầy ...
384,433.0,Những bậc đá chạm mây,"Ngày xưa, dưới chân núi Hồng Lĩnh có một xóm n...",NaN,NaN,NaN,NaN,NaN,Lớp 3,CÁNH DIỀU,"Thấy lên núi phải đi đường vòng, ông bàn với m...","Ngày xưa, dưới chân núi Hồng Lĩnh có một xóm n...","Ngày xưa, dưới chân núi Hồng Lĩnh có một xóm n..."


| Model     | Level 1 | Level 2 | Level 3 | Level 4 | Level 5 | Cổ tích               | Bài học đời sống |
| --------- | ------- | ------- | ------- | ------- | ------- | --------------------- | ---------------- |
| **Lớp 1** | 50%     | 30%     | 0%      | 0%      | 0%      | 20% (đã lọc đơn giản) | 0%               |
| **Lớp 2** | 30%     | 40%     | 15%     | 0%      | 0%      | 15%                   | 0%               |
| **Lớp 3** | 0%      | 30%     | 40%     | 20%     | 0%      | 10%                   | 0%               |
| **Lớp 4** | 0%      | 0%      | 30%     | 40%     | 15%     | 10%                   | 5%               |
| **Lớp 5** | 0%      | 0%      | 20%     | 30%     | 30%     | 10%                   | 10%              |


In [ ]:
llm = ChatOllama(model="myaniu/qwen2.5-1m")  # hoặc "llama3:8b-instruct"

def extract_lesson(text, grade="1"):
    prompt = [
        SystemMessage(content=f"Bạn là giáo viên dạy môn học tiếng Việt cho học sinh lớp {grade} tại Việt Nam."),
        HumanMessage(content=f"""
        Đọc đoạn văn sau và rút ra bài học phù hợp với học sinh lớp {grade}.
        Viết ngắn gọn, dễ hiểu, trong chỉ 1 câu.

        ĐOẠN VĂN:
        {text}

        KẾT QUẢ:
        """)
    ]

    response = llm(prompt)
    return response.content.strip()

text = """Ngày khai trường đã đến. Sáng sớm, mẹ mới gọi một câu mà tôi đã vùng dậy, khác hẳn mọi ngày. Loáng một cái, tôi đã chuẩn bị xong mọi thứ. Bố ngạc nhiên nhìn tôi, còn mẹ cười tủm tỉm. Tôi ríu rít: “Con muốn đến sớm nhất lớp.” Tôi háo hức tưởng tượng ra cảnh mình đến đầu tiên, cất tiếng chào thật to những bạn đến sau. Nhưng vừa đến cổng trường, tôi đã thấy mấy bạn cùng lớp đang ríu rít nói cười ở trong sân. Thì ra, không chỉ mình tôi muốn đến sớm nhất. Tôi chào mẹ, chạy ào vào cùng các bạn. Chúng tôi tranh nhau kể về chuyện ngày hè. Ngay cạnh chúng tôi, mấy em lớp 1 đang rụt rè níu chặt tay bố mẹ, thật giống tôi năm ngoái. Trước các em, tôi cảm thấy mình lớn lắm. Tôi đã là học sinh lớp 2 rồi cơ mà. """
print(extract_lesson(text, grade="2"))


In [ ]:
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 13.6 MB/s eta 0:00:00


In [ ]:

import os
from google.cloud import vision

# ✅ Thiết lập credentials đúng file JSON của bạn
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/credentials/projectsummarize-ee4dd5a1f9e8.json"

# 🔹 Khởi tạo client
client = vision.ImageAnnotatorClient()

# 🔹 Hàm OCR
def google_vision_ocr(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    return response.full_text_annotation.text

# ✅ Test OCR với ảnh của bạn
image_path = "/content/drive/MyDrive/images/Screenshot 2025-10-04 154203.png"  # thay nếu khác
text = google_vision_ocr(image_path)

print("===== Kết quả OCR =====")
print(text)

# ✅ Lưu ra file
with open("/content/drive/MyDrive/export_to_text/vision_text.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("✅ Đã lưu vào /content/drive/MyDrive/export_to_text/vision_text.txt")


===== Kết quả OCR =====
Bài Mở đầu
Em là học sinh
Em việt
Em nói
Em doc
Đồ dùng học tập
Đồ dùng học tập
Em làm việc nhóm
Em trải nghiêm
Em học ở nhà
✅ Đã lưu vào /content/drive/MyDrive/export_to_text/vision_text.txt


In [ ]:
import os
from google.cloud import vision

# Đảm bảo Drive đã được mount và credentials đã set
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/credentials/projectsummarize-ee4dd5a1f9e8.json"

client = vision.ImageAnnotatorClient()

def google_vision_ocr(image_path):
    with open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    return response.full_text_annotation.text

# 🔹 Thư mục chứa ảnh
image_folder = "/content/drive/MyDrive/images/"
output_file = "/content/drive/MyDrive/export_to_text/vision_text.txt"

# 🔹 Mở file để ghi (chế độ 'a' = append thêm nội dung mới)
with open(output_file, "a", encoding="utf-8") as f:
    for filename in os.listdir(image_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(image_folder, filename)
            print(f"🔍 Đang OCR ảnh: {filename}")
            text = google_vision_ocr(image_path)

            # Ghi vào file với tiêu đề ảnh
            f.write(f"\n===== ẢNH: {filename} =====\n")
            f.write(text + "\n")

print("✅ Hoàn thành OCR toàn bộ ảnh và lưu vào vision_text.txt!")


🔍 Đang OCR ảnh: Screenshot 2025-10-04 154311.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154341.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154407.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154433.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154532.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154821.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154832.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154845.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154857.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 154904.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155027.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155033.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155038.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155043.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155048.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155052.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155058.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155102.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155107.png
🔍 Đang OCR ảnh: Screenshot 2025-10-04 155111.png
🔍 Đang OCR ảnh: Scre

In [ ]:
!wget -q https://raw.githubusercontent.com/undertheseanlp/dictionary/master/dictionaries/tudientv/words.txt -O vietnamese_dict.txt

In [ ]:
!pip install -q google-generativeai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# =====================
# Colab script: Sửa dấu + sửa lỗi nhẹ bằng từ điển (offline)
# =====================

# Yêu cầu: đã mount Drive trước đó

import json
import unicodedata
import re
from collections import defaultdict
from tqdm import tqdm

# ---------------------
# Cấu hình đường dẫn
# ---------------------
DICT_PATH = "/content/drive/MyDrive/vietnamese_dict.txt"   # file JSONL bạn đã có
INPUT_FILE = "/content/drive/MyDrive/export_to_text/vision_text.txt"
OUTPUT_FILE = "/content/drive/MyDrive/export_to_text/vision_text_fixed.txt"

# ---------------------
# Hàm tiện ích: remove accents (chuẩn hoá không dấu)
# ---------------------
def remove_accents(s: str) -> str:
    # chuẩn hoá NFD rồi loại bỏ các ký tự combining
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if unicodedata.category(ch) != 'Mn')
    # chuyển một số ký tự đặc biệt về chuẩn ascii
    s = s.replace('đ', 'd').replace('Đ', 'D')
    return unicodedata.normalize('NFC', s)

# ---------------------
# Hàm Levenshtein với ngưỡng dừng sớm (early stop)
# ---------------------
def levenshtein_with_max(a: str, b: str, max_dist: int):
    # returns distance if <= max_dist, else returns a value > max_dist
    # optimized: use two-row DP
    if abs(len(a) - len(b)) > max_dist:
        return max_dist + 1
    if len(a) < len(b):
        a, b = b, a
    previous = list(range(len(b) + 1))
    for i, ca in enumerate(a, start=1):
        current = [i] + [0] * len(b)
        # compute current row with pruning
        min_in_row = current[0]
        for j, cb in enumerate(b, start=1):
            insert_cost = current[j-1] + 1
            delete_cost = previous[j] + 1
            replace_cost = previous[j-1] + (0 if ca == cb else 1)
            current[j] = min(insert_cost, delete_cost, replace_cost)
            if current[j] < min_in_row:
                min_in_row = current[j]
        if min_in_row > max_dist:
            return max_dist + 1
        previous = current
    return previous[-1]

# ---------------------
# Load từ điển JSONL -> tập các từ hợp lệ + map không dấu -> list từ
# ---------------------
vietnamese_words = set()
deaccent_map = defaultdict(list)

print("⏳ Đang load từ điển từ:", DICT_PATH)
with open(DICT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            item = json.loads(line)
            word = item.get("text", "").strip()
            if not word:
                continue
            # chuẩn hoá: chữ thường
            key = word.lower()
            # chỉ giữ nếu có chữ (có thể chứa space cho cụm từ)
            # loại bỏ những dòng có kí tự lạ ở đầu
            if any(ch.isalpha() for ch in key):
                vietnamese_words.add(key)
                deacc = remove_accents(key).lower()
                deaccent_map[deacc].append(key)
        except Exception as e:
            # bỏ qua dòng lỗi
            continue

print("✅ Đã load từ điển. Số từ (cụm từ) trong bộ từ điển:", len(vietnamese_words))
print("Ví dụ 20 từ:", list(vietnamese_words)[:20])

# ---------------------
# Tokenize đơn giản (giữ punctuation để phục hồi)
# ---------------------
token_split_re = re.compile(r"(\w+|[^\w\s])", flags=re.UNICODE)

def smart_tokenize(line: str):
    # trả về list token, giữ nguyên dấu câu là token riêng
    return token_split_re.findall(line)

def is_number_only(s: str):
    s_stripped = s.strip()
    # nếu chỉ có số hoặc chỉ chứa số+punct
    return bool(re.fullmatch(r"[\d\W]+", s_stripped)) and any(ch.isdigit() for ch in s_stripped)

# ---------------------
# Hàm sửa 1 token bằng từ điển
# ---------------------
def correct_token(token: str, max_lev=2):
    """
    token: token nguyên bản (có thể kèm hoa thường)
    trả về token đã sửa (giữ nguyên kiểu hoa)
    logic:
      1. nếu token trong dict -> giữ
      2. thử tìm trong deaccent_map (khớp không dấu)
      3. tìm từ gần nhất bằng levenshtein trong dictionary nhưng chỉ xét từ có độ dài tương đương
    """
    orig = token
    # giữ nguyên nếu token là punctuation
    if not any(ch.isalpha() for ch in token):
        return token

    # tách prefix/suffix punctuation (ví dụ: "Em," -> 'Em' và ',')
    prefix = re.match(r'^[^\w]*', token).group(0) if re.match(r'^[^\w]*', token) else ''
    suffix = re.search(r'[^\w]*$', token).group(0) if re.search(r'[^\w]*$', token) else ''
    core = token[len(prefix): len(token)-len(suffix) if suffix else None]

    if not core:
        return token

    # remember capitalization
    is_title = core.istitle()
    is_upper = core.isupper()

    core_low = core.lower()

    # 1) exact match
    if core_low in vietnamese_words:
        corrected = core_low
    else:
        # 2) match via deaccent map
        de = remove_accents(core_low)
        candidates = deaccent_map.get(de, [])
        if candidates:
            # choose candidate with minimal levenshtein to core_low (with limit)
            best = None
            bestd = max_lev + 10
            for c in candidates:
                d = levenshtein_with_max(core_low, c, max_lev)
                if d <= max_lev and d < bestd:
                    best = c
                    bestd = d
            if best:
                corrected = best
            else:
                # if none within max_lev, choose first candidate as fallback
                corrected = candidates[0]
        else:
            # 3) fallback: scan dictionary for nearby words (expensive but limited by length)
            best = None
            bestd = max_lev + 10
            L = len(core_low)
            # iterate through words of similar length only to reduce cost
            # choose lengths L-1, L, L+1
            for w in vietnamese_words:
                if abs(len(w) - L) > 1:
                    continue
                d = levenshtein_with_max(core_low, w, max_lev)
                if d <= max_lev and d < bestd:
                    best = w
                    bestd = d
                    if bestd == 0:
                        break
            if best:
                corrected = best
            else:
                # không tìm được -> giữ nguyên
                corrected = core_low

    # restore capitalization similar to original
    if is_upper:
        out = corrected.upper()
    elif is_title:
        out = corrected.capitalize()
    else:
        out = corrected

    return prefix + out + suffix

# ---------------------
# Xử lý toàn file
# ---------------------
print("⏳ Bắt đầu xử lý file:", INPUT_FILE)
with open(INPUT_FILE, "r", encoding="utf-8") as fin, \
     open(OUTPUT_FILE, "w", encoding="utf-8") as fout:
    for raw_line in tqdm(fin):
        line = raw_line.rstrip("\n")
        stripped = line.strip()

        # Giữ nguyên các dòng định dạng ảnh
        if stripped.startswith("=====") and "ẢNH" in stripped:
            fout.write(line + "\n")
            continue

        # Bỏ các dòng chỉ toàn số / ký tự rác
        if is_number_only(stripped):
            # bỏ hoàn toàn (hoặc có thể ghi newline nếu muốn giữ vị trí)
            # chúng ta ghi 1 dòng rỗng để giữ đôi chút format
            fout.write("\n")
            continue

        if stripped == "":
            fout.write("\n")
            continue

        # Tokenize và sửa từng token
        tokens = smart_tokenize(line)
        corrected_tokens = [correct_token(t) for t in tokens]
        new_line = "".join(
            # smart join: nếu token là word and next token is word, insert space
            # but tokenization keeps words and punctuation separate; we'll rebuild with spaces when needed
            corrected_tokens[i] + ("" if (i+1 < len(corrected_tokens) and re.match(r'[^\w\s]', corrected_tokens[i+1])) else " ")
            for i in range(len(corrected_tokens))
        ).rstrip()

        fout.write(new_line + "\n")

print("✅ Hoàn tất! File kết quả lưu tại:", OUTPUT_FILE)


⏳ Đang load từ điển từ: /content/drive/MyDrive/vietnamese_dict.txt
✅ Đã load từ điển. Số từ (cụm từ) trong bộ từ điển: 36531
Ví dụ 20 từ: ['hoa tiên', 'barie', 'chao chát', 'mạch máu', 'bá chủ', 'ơn đức', 'kĩ càng', 'thao', 'ruộng nương', 'cây nông nghiệp', 'kênh kiệu', 'hoà giải', 'hz', 'vàng hươm', 'đắt cắt ra miếng', 'thầy chùa', 'duyệt y', 'phạm phòng', 'từ vựng', 'thiết bị ngoại vi']
⏳ Bắt đầu xử lý file: /content/drive/MyDrive/export_to_text/vision_text.txt


6743it [00:27, 242.47it/s]

✅ Hoàn tất! File kết quả lưu tại: /content/drive/MyDrive/export_to_text/vision_text_fixed.txt


In [12]:
import json

dict_path = "/content/drive/MyDrive/vietnamese_dict.txt"  # ✅ Đường dẫn bạn cung cấp

vietnamese_words = set()
with open(dict_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line.strip())
            word = item.get("text", "").strip().lower()
            if word and word.replace(" ", "").isalpha():  # Cho phép cụm từ có dấu cách
                vietnamese_words.add(word)
        except:
            pass

print("✅ Tổng số từ trong từ điển:", len(vietnamese_words))
print(list(vietnamese_words)[:20])  # xem thử

# ==========================================
# 3️⃣ Hàm lọc từng dòng
# ==========================================
def clean_line(line):
    words = line.split()
    cleaned = [w for w in words if w.lower().strip(".,!?;:()\"'") in vietnamese_words]
    return " ".join(cleaned)

# ==========================================
# 4️⃣ Đọc file OCR gốc → tạo file sạch
# ==========================================
input_file = "/content/drive/MyDrive/export_to_text/vision_text_fixed.txt"
output_file = "/content/drive/MyDrive/export_to_text/vision_text_clean.txt"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        cleaned = clean_line(line)
        fout.write(cleaned + "\n")

print("✅ Đã lọc xong! File mới tại:", output_file)


✅ Tổng số từ trong từ điển: 36167
['đơn côi', 'đồng bằng', 'lờm lợm', 'giữ ý giữ tứ', 'cấy hái', 'phosphorite', 'mồ côi mồ cút', 'chăm sóc', 'nhấn', 'hộp thoại', 'đoan', 'lũn', 'án ngữ', 'cành cạch', 'giá rét', 'xe pháo', 'an toạ', 'đêm', 'múi chiếu bản đồ', 'chế']
✅ Đã lọc xong! File mới tại: /content/drive/MyDrive/export_to_text/vision_text_clean.txt
